<a href="https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這支 colab notebook 是為了檢索 [news-digest](https://news-digest.vercel.app/) 網站上的新聞而寫的。可以執行多條件檢索，執行步驟如下：

1. 按下 Install 下那個 code cell 前面的 ► 按鈕來安裝用到的模組
2. 依喜好執行 UI only on Jupyter widgets 或 UI mixed colab forms and Jupyter widgets

### Install

In [ ]:
import os

fns = ['clip.py', 'line.py', 'op.py', 'hashtag.py']

for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
for fn in fns:
    url = f'https://raw.githubusercontent.com/YorkJong/news-digest/main/api/{fn}'
    !wget $url

import clip
import line
import hashtag

#-------------------------------------------------------------------------------

import ipywidgets as widgets

def create_menu(name, options):
    m = widgets.SelectMultiple(
        options=options,
        #value=[options[0]],
        rows=min(len(options), 20),
        description=name,
        disabled=False
    )
    return m

def create_widgets_only_ui():
    uiDays = widgets.Dropdown(
        options=[1, 2, 7, 14, 30],
        value=1,
        description='Period (days)',
        disabled=False,
    )
    content = clip.merge_recent_journals(uiDays.value)
    uiCates = create_menu(name='Categories', options=clip.get_categories(content))

    uiTags = create_menu(name='Hash-Tags', options=[])

    uiOp = widgets.RadioButtons(
        options=[
            ('ANY selected tags', hashtag.get_lines_with_any_hashtags),  
            ('ALL selected tags', hashtag.get_lines_with_all_hashtags)
        ],
        value=hashtag.get_lines_with_any_hashtags,
        description='query with',
        disabled=False
    )

    with_hashtags = widgets.Checkbox(
        value=True,
        description='show hashtags',
        disabled=False,
    )

    with_headers = widgets.Checkbox(
        value=True,
        description='show headers',
        disabled=False,
    )

    vbox = widgets.VBox([uiOp, with_hashtags, with_headers])
    ui = widgets.HBox([uiDays, uiCates, uiTags, vbox])
    output = widgets.Output()

    def on_days_change(change):
        nonlocal content
        content = clip.merge_recent_journals(uiDays.value)
        uiCates.options = clip.get_categories(content)
    uiDays.observe(on_days_change, names='value')

    def on_cates_change(change):
        lines = clip.get_lines_of_categories(uiCates.value, content, True, True)
        uiTags.options=hashtag.get_hashtags(lines)
        uiTags.rows=min(len(uiTags.options), 20)
    uiCates.observe(on_cates_change, names='value')

    def on_tags_change(change):
        lines = clip.get_lines_of_categories(uiCates.value, content, True, True)
        text = '\n'.join(uiOp.value(lines, uiTags.value))
        lines = clip.get_lines_of_categories(uiCates.value, text, with_hashtags.value, with_headers.value)
        output.clear_output()
        with output:
            print('\n'.join(lines))
    uiTags.observe(on_tags_change, names='value')
    uiOp.observe(on_tags_change, names='value')
    with_hashtags.observe(on_tags_change, names='value')
    with_headers.observe(on_tags_change, names='value')

    display(ui, output)

### Query (UI only on Jupyter widgets)

In [8]:
create_widgets_only_ui()

Output()

### Query (UI mixed forms and Jupyter widgets)
- 依序選取，並執行查詢條件：週期（days）, 分類（Categories）, Tags, with ANY/ALL tags

In [3]:
#@title Step 1. pick a **period** { run: "auto", display-mode: "form" }
days = 1 #@param ["1", "2", "7", "14", "30"] {type:"raw", allow-input: true}
content = clip.merge_recent_journals(days)

#### Step 2. select **Categories**
- 先按下面那個 code cell 前面的 ► 按鈕，以列出完整的分類(Catetories)選單
- macOS 下操作可按Cmd後用滑鼠複選（Windows 系統則按下Ctrl鍵後以滑鼠複選）
  - 全選可按 Cmd+A (macOS) 或者 Ctrl+A (Windows)

In [4]:
categories = create_menu(name='Categories', options=clip.get_categories(content))
display(categories)

SelectMultiple(description='Categories', options=('Tesla & SpaceX; Vehicle', 'Tech Titans', 'Finance', 'Taiwan…

#### Step 3. select hash **Tags**
- 先按下面那個 code cell 前面的 ► 按鈕，以列出完整的 #Tag 選單
- macOS 下操作可按Cmd後用滑鼠複選（Windows 系統則按下Ctrl鍵後以滑鼠複選）
   - 全選可按 Cmd+A (macOS) 或者 Ctrl+A (Windows)

In [5]:
lines = clip.get_lines_of_categories(categories.value, content, with_hashtags=True, with_headers=True)
tags = create_menu(name='Hash-Tags', options=hashtag.get_hashtags(lines))
display(tags)

SelectMultiple(description='Hash-Tags', options=('#China', '#FCEV'), rows=2, value=())

In [ ]:
#@title Step 4. decide **ANY** (or **ALL**) hash-tags with { run: "auto", display-mode: "form" }
hashtag_with = "ALL" #@param ["ANY", "ALL"]

tbl = {
    'ANY': hashtag.get_lines_with_any_hashtags,
    'ALL': hashtag.get_lines_with_all_hashtags,
}

out_lines = tbl[hashtag_with](lines, tags.value)
print('\n'.join(out_lines))

### See Also
- [news_clip.ipynb (from GitHub)](https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_clip.ipynb)